In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve,roc_auc_score


from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import StandardScaler,PowerTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

pd.options.mode.chained_assignment = None 

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.head()

## EDA

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
# features distributions
data.hist(figsize=(15,10));

In [ ]:
# Replace 0 in glucose,blood pressure,skinthinkness,insulin => nulls
data[['Glucose','BloodPressure','BloodPressure','SkinThickness','Insulin','BMI']] = data[['Glucose','BloodPressure','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
data.isnull().sum()

In [ ]:
data  = data.drop(['Insulin','SkinThickness'],axis = 1)

In [ ]:
# correlation between features and outcome
fig , ax = plt.subplots(1,1,figsize = (7,7))
corr = data.corr()
sns.heatmap(corr,annot = True, ax=ax)

In [ ]:
sns.pairplot(data = data,hue = 'Outcome')

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
sns.boxplot(data=data,ax=ax)
ax.tick_params(axis='x', rotation=90);

Ideas :<br>
1. impute missing values in glucose,bloodpressure,bmi by mean or median (due to the given distribution)
2. when features scaling , we can use log scale as some features follow some log trend and others are normal so wont be affected much.
3. log scale can fix most of the outliers in the data as most of the features with outliers follow some log trend.
4. grouped average by label for nulls, but this will cause data leakage , it will give high scores but not recomended.

## Preprocessing

In [ ]:
averages_0 = pd.DataFrame(data[data['Outcome']==0].mean(axis=0)).T
averages_1 = pd.DataFrame(data[data['Outcome']==1].mean(axis=0)).T
averages_0

In [ ]:
# possible imputation but will cause data leakage
# it will increase the scores but data leakage will be a huge proplem
def grouped_average(X,y,av_0,av_1,cols,target):
    df = pd.merge(X,y,left_index=True,right_index=True)
    for col in cols:
        df[col].loc[(df[col].isnull()) & (df[target]==1)] = av_0[col].values[0]
        df[col].loc[(df[col].isnull()) & (df[target]==0)] = av_1[col].values[0]
    return df

In [ ]:
# train test split

X = data.drop(['Outcome'],axis = 1)
y = data['Outcome']
col_names = X.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


impute = KNNImputer(n_neighbors = 8)
impute.fit(X_train)

X_train = pd.DataFrame(impute.transform(X_train),columns = col_names)
X_test = pd.DataFrame(impute.transform(X_test),columns = col_names)

X_train.isnull().sum()

In [ ]:
# Scaling the data
scaler = PowerTransformer()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train),columns = col_names)
X_test = pd.DataFrame(scaler.transform(X_test),columns = col_names)

X_train.hist(figsize = (10,10));

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
sns.boxplot(data=X_train,ax=ax)
ax.tick_params(axis='x', rotation=90);

we can comeback later and see the effect of removing the small number of remaining outliers.

## Applying models

### 1-Logistic regression

In [ ]:
model = LogisticRegression(class_weight = 'balanced',random_state = 42)
model.fit(X_train,y_train)
y_pred_lr = model.predict(X_test)
print(classification_report(y_test,y_pred_lr))
roc = roc_auc_score(y_test,y_pred_lr)
print('roc score : ',roc)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))
matrix_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(matrix_lr,annot=True,fmt='g',ax=ax[0])
plot_roc_curve(model,X_test,y_test,ax=ax[1]);

ax[0].title.set_text('confusion matrix')
ax[1].title.set_text('ROC curve')

ax[0].set_xlabel('predicted')
ax[0].set_ylabel('actual');


### 2- KNN

In [ ]:
# KNN with grid search
model_knn = KNeighborsClassifier()
grid = {
    'n_neighbors':[i+1 for i in range(30)],
    'weights':['uniform','distance'],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']
}

knn_gs = GridSearchCV(model_knn,grid,cv=5,scoring = 'roc_auc')
knn_gs.fit(X_train,y_train)

print(knn_gs.best_estimator_)
print(knn_gs.best_score_)


In [ ]:
y_pred_k = knn_gs.best_estimator_.predict(X_test)
print(classification_report(y_test,y_pred_k))

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))
matrix_knn = confusion_matrix(y_test, y_pred_k)
sns.heatmap(matrix_knn,annot=True,fmt='g',ax=ax[0])
plot_roc_curve(knn_gs.best_estimator_,X_test,y_test,ax=ax[1]);

ax[0].title.set_text('confusion matrix')
ax[1].title.set_text('ROC curve')

ax[0].set_xlabel('predicted')
ax[0].set_ylabel('actual');

### 3-SVC

In [ ]:
# SVC using grid search
model_svc = SVC(class_weight='balanced')
grid = {
    'C':[i+1 for i in range(30)],
    'kernel':['rbf','poly'],
    'degree':[1,2,3]
}

svc_gs = GridSearchCV(model_svc,grid,cv=5,scoring = 'roc_auc')
svc_gs.fit(X_train,y_train)

print(svc_gs.best_estimator_)
print(svc_gs.best_score_)


In [ ]:
y_pred_s = svc_gs.best_estimator_.predict(X_test)
print(classification_report(y_test,y_pred_s))

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))
matrix_svc = confusion_matrix(y_test, y_pred_s)
sns.heatmap(matrix_svc,annot=True,fmt='g',ax=ax[0])
plot_roc_curve(svc_gs.best_estimator_,X_test,y_test,ax=ax[1]);

ax[0].title.set_text('confusion matrix')
ax[1].title.set_text('ROC curve')

ax[0].set_xlabel('predicted')
ax[0].set_ylabel('actual');

### Scores

In [ ]:
print("SVC score roc: ",svc_gs.best_score_)
print("Knn score roc: ",knn_gs.best_score_)
print("Logistic regression score roc: ",roc)

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(20,10))

print("SVC score roc: ",svc_gs.best_score_)
print("Knn score roc: ",knn_gs.best_score_)
print("Logistic regression score roc: ",roc)

plot_roc_curve(svc_gs.best_estimator_,X_test,y_test,ax=ax[0,0])
plot_roc_curve(knn_gs.best_estimator_,X_test,y_test,ax=ax[0,1])
plot_roc_curve(model,X_test,y_test,ax=ax[0,2])

sns.heatmap(matrix_svc,annot=True,fmt='g',ax=ax[1,0])
sns.heatmap(matrix_knn,annot=True,fmt='g',ax=ax[1,1])
sns.heatmap(matrix_lr,annot=True,fmt='g',ax=ax[1,2])


ax[0,0].title.set_text('SVC')
ax[0,1].title.set_text('KNN')
ax[0,2].title.set_text('Logistic regression')

ax[1,0].set_xlabel('predicted')
ax[1,0].set_ylabel('actual');

ax[1,1].set_xlabel('predicted')
ax[1,1].set_ylabel('actual');

ax[1,2].set_xlabel('predicted')
ax[1,2].set_ylabel('actual');